## Installing Libraries Needed

In [1]:
!python -m pip install pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!python -m pip install openpyxl



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Dataset Inventory Loader

In [3]:
import os
import re

base_path = r"G:\My Drive\Labor Force Survey"

# Month ordering
month_order = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

# Patterns
month_pattern = re.compile(
    r"(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)",
    re.IGNORECASE
)
year_pattern = re.compile(r"(20\d{2})")

# Detect year folders from drive
year_folders = [
    f for f in os.listdir(base_path)
    if os.path.isdir(os.path.join(base_path, f)) and f.isdigit()
]

print("Detected year folders:", sorted(year_folders))

inventory = {}

for year in sorted(year_folders):
    year_path = os.path.join(base_path, year)

    # Accept both CSV and XLSX
    data_files = [
        f for f in os.listdir(year_path)
        if f.lower().endswith(".csv") or f.lower().endswith(".xlsx")
    ]

    inventory[year] = {}

    for file in data_files:
        upper = file.upper()

        # Detect type
        if upper.endswith(".XLSX"):
            filetype = "metadata"  # XLSX = metadata
        else:
            filetype = "survey"    # CSV = survey

        # Detect month
        month_match = month_pattern.search(upper)
        month = (
            month_match.group(1).capitalize()
            if month_match
            else "Unmatched"
        )

        # Detect year inside filename
        year_match = year_pattern.search(upper)
        file_year = year_match.group(1) if year_match else "UNKNOWN"

        # Store into inventory
        if month not in inventory[year]:
            inventory[year][month] = []

        inventory[year][month].append({
            "filename": file,
            "filetype": filetype,
            "file_year": file_year
        })

# Print clean summary
print("\n=== DATASET INVENTORY SUMMARY ===\n")

for yr in sorted(inventory.keys()):
    print(f"Year {yr}:")

    sorted_months = sorted(
        inventory[yr].keys(),
        key=lambda m: month_order.get(m, 99)
    )

    for month in sorted_months:
        print(f"  {month}:")
        for item in inventory[yr][month]:
            print(f"    {item['filename']} ({item['filetype']})")

    print()


Detected year folders: ['2018', '2019', '2022', '2023', '2024']

=== DATASET INVENTORY SUMMARY ===

Year 2018:
  January:
    JANUARY_2018_METADATA.xlsx (metadata)
    JANUARY_2018.CSV (survey)
  April:
    APRIL_2018.CSV (survey)
    APRIL_2018_METADATA.xlsx (metadata)
  July:
    JULY_2018.CSV (survey)
    JULY_2018_METADATA.xlsx (metadata)
  October:
    OCTOBER_2018.CSV (survey)
    OCTOBER_2018_METADATA.xlsx (metadata)

Year 2019:
  January:
    JANUARY_2019.CSV (survey)
    JANUARY_2019_METADATA.xlsx (metadata)
  April:
    APRIL_2019.CSV (survey)
    APRIL_2019_METADATA.xlsx (metadata)
  July:
    JULY_2019.CSV (survey)
    JULY_2019_METADATA.xlsx (metadata)
  October:
    OCTOBER_2019.CSV (survey)
    OCTOBER_2019_METADATA.xlsx (metadata)

Year 2022:
  January:
    JANUARY_2022.csv (survey)
    JANUARY_2022_METADATA.xlsx (metadata)
  February:
    FEBRUARY_2022.csv (survey)
    FEBRUARY_2022_METADATA.xlsx (metadata)
  March:
    MARCH_2022.csv (survey)
    MARCH_2022_METADATA.x

## Load Dataset Function

In [4]:
def load_dataset(year, month, filetype="survey", sheet_number=None):
    """
    Load a dataset from the inventory.

    year: str, e.g., "2018"
    month: str, e.g., "January"
    filetype: "survey" or "metadata"
    sheet_number: 0(sheet 1) or 1(sheet 2)
    """
    file_info = next(
        (f for f in inventory[year][month] if f["filetype"] == filetype),
        None
    )
    if not file_info:
        raise ValueError(f"No {filetype} file found for {month} {year}")

    file_path = os.path.join(base_path, year, file_info["filename"])
    
    if filetype == "survey":
        return pd.read_csv(file_path, low_memory=False)
    
    if sheet_number is not None:
        return pd.read_excel(file_path, sheet_name=sheet_number)
    
    return pd.read_excel(file_path)

## Metadata Sheet 1 Function 

In [5]:
def extract_variables(df):
    """
    Extract variable names and descriptions from metadata DataFrame.
    Looks for values in columns 'Unnamed: 4' (variable) and 'Unnamed: 5' (description).
    Returns a clean DataFrame with ['Variable', 'Description'].
    """

    # Keep only rows where column 4 has a variable name
    filtered = df[df['Unnamed: 4'].notna()][['Unnamed: 4', 'Unnamed: 5']]

    # Rename columns
    filtered.columns = ['Variable', 'Description']

    return filtered


In [6]:
import pandas as pd

# Load metadata
January_metadata = load_dataset("2018", "January", "metadata", 0)

# Call your function
variables_df = extract_variables(January_metadata)

# View results
variables_df.head()


,Variable,Description
1,PUFREG,Region
2,PUFPRV,Province
3,PUFPRRCD,Province Recode
4,PUFHHNUM,Household Unique Sequential Number
6,PUFURB2K10,2010Urban-RuralFIES


## Metadata Sheet 2 Function

In [7]:
# Load metadata
January_metadata = load_dataset("2018", "January", "metadata", 1)

January_metadata.head()

,PUFREG_VS1,Region,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,National Capital Region,13,NaN,NaN
1,NaN,NaN,Cordillera Administrative Region,14,NaN,NaN
2,NaN,NaN,Region I - Ilocos Region,1,NaN,NaN
3,NaN,NaN,Region II - Cagayan Valley,2,NaN,NaN
4,NaN,NaN,Region III - Central Luzon,3,NaN,NaN


In [8]:
import pandas as pd

def reshape_sheet2_friendly(df):
    reshaped = []
    max_value_tracker = {}

    # Fill NaNs for easier handling
    df = df.fillna('').astype(str)

    current_var = None
    current_desc = None

    # Extract the first variable and description from the header row
    first_var = df.columns[0].strip() if df.columns[0] else 'UnknownVar'
    first_desc = df.columns[1].strip() if df.columns[1] else None

    for idx, row in df.iterrows():
        var_candidate = row.iloc[0].strip() or None
        desc_candidate = row.iloc[1].strip() or None
        label = row.iloc[2].strip() or None

        # Update current variable only if present
        current_var = var_candidate or current_var or first_var
        current_desc = desc_candidate or current_desc or first_desc

        max_value_tracker.setdefault(current_var, 0)

        # Skip rows without a label
        if not label:
            continue

        raw_value_start = row.iloc[3].strip() or None
        raw_value_end = row.iloc[4].strip() or None
        additional_value = row.iloc[5].strip() or None

        # Determine value for min/max
        try:
            value_for_calc = int(raw_value_start) if raw_value_start else max_value_tracker[current_var] + 1
        except (ValueError, TypeError):
            value_for_calc = max_value_tracker[current_var] + 1

        max_value_tracker[current_var] = max(max_value_tracker[current_var], value_for_calc)

        try:
            min_value = int(value_for_calc)
        except (ValueError, TypeError):
            min_value = None

        try:
            max_value = int(raw_value_end) if raw_value_end else min_value
        except (ValueError, TypeError):
            max_value = min_value

        reshaped.append({
            "Variable": current_var,
            "Description": current_desc,
            "Label": label,
            "min_value": min_value,
            "max_value": max_value,
            "additional_value": additional_value
        })

    return pd.DataFrame(reshaped)

In [9]:
sheet2 = load_dataset("2018", "January", "metadata", 1)
clean = reshape_sheet2_friendly(sheet2)
clean.head(30)

,Variable,Description,Label,min_value,max_value,additional_value
0,PUFREG_VS1,Region,National Capital Region,13,13,None
1,PUFREG_VS1,Region,Cordillera Administrative Region,14,14,None
2,PUFREG_VS1,Region,Region I - Ilocos Region,1,1,None
3,PUFREG_VS1,Region,Region II - Cagayan Valley,2,2,None
4,PUFREG_VS1,Region,Region III - Central Luzon,3,3,None
5,PUFREG_VS1,Region,Region IVA - CALABARZON,4,4,None
6,PUFREG_VS1,Region,Region IVB - MIMAROPA,17,17,None
7,PUFREG_VS1,Region,Region V- Bicol,5,5,None
8,PUFREG_VS1,Region,Region VI - Western Visayas,6,6,None
9,PUFREG_VS1,Region,Region VII - Central Visayas,7,7,None


In [10]:
sheet2 = load_dataset("2024", "August", "metadata", 1)
clean = reshape_sheet2_friendly(sheet2)
clean.head(30)

,Variable,Description,Label,min_value,max_value,additional_value
0,PUFSVYMO_VS1,Survey Month,January,1,1,None
1,PUFSVYMO_VS1,Survey Month,February,2,2,None
2,PUFSVYMO_VS1,Survey Month,March,3,3,None
3,PUFSVYMO_VS1,Survey Month,April,4,4,None
4,PUFSVYMO_VS1,Survey Month,May,5,5,None
5,PUFSVYMO_VS1,Survey Month,June,6,6,None
6,PUFSVYMO_VS1,Survey Month,July,7,7,None
7,PUFSVYMO_VS1,Survey Month,August,8,8,None
8,PUFSVYMO_VS1,Survey Month,September,9,9,None
9,PUFSVYMO_VS1,Survey Month,October,10,10,None
